In [1]:
import re
import requests
import sqlite3
from bs4 import BeautifulSoup
qy = open('db.txt',mode='a',encoding='utf-8')#这里是要存入的文件目录
movies = []
## 由于网页的分页机制，每次只能获取25条结果,这里设置为获取前50的结果，因此需要循环两次
for i in range(0,2):
    headers = {#这里模拟浏览器进行访问
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.82 Safari/537.36',
        'Host': 'movie.douban.com'
    }     
    res = 'https://movie.douban.com/top250?start='+str(25*i)#25次
    r = requests.get(res, headers=headers, timeout=10)#设置超时时间
    soup = BeautifulSoup(r.text, "html.parser")#设置解析方式,也可以使用其他方式。
    div_list = soup.find_all('div', class_='item')
    for each in div_list:
        movie = {}
        moviename = each.find('div', class_='hd').a.span.text.strip()
        movie['title'] = moviename
        rank = each.find('div', class_='pic').em.text.strip()
        movie['rank'] = rank
        info = each.find('div', class_='bd').p.text.strip()
        info = info.replace('\n', "")
        info = info.replace(" ", "")
        info = info.replace("\xa0", "")
        director = re.findall(r'[导演:].+[主演:]', info)[0]
        director = director[3:len(director) - 3]
        movie['director'] = director
        release_date = re.findall(r'[0-9]{4}', info)[0]
        movie['release_date'] = release_date
        plot = re.findall(r'[0-9]*[/].+[/].+', info)[0]
        plot = plot[1:]
        #print(plot)
        plot = plot[plot.index('/') + 1:]
        #print(plot)
        plot = plot[plot.index('/') + 1:]
        #print(plot)
        movie['plot'] = plot
        star = each.find('div', class_='star')
        star = star.find('span', class_='rating_num').text.strip()
        movie['star'] = star
        movies.append(movie)
        print(movie,file=qy)#保存到文件中
con = sqlite3.connect('movie_review.db')
print('连接成功->')
print('开始创建表->')
cursor= con.cursor()
cursor.execute("""create table if not exists douban 
                ( title char(40),
                  ranks char(40),
                  director char(40),
                  release_date char(40), 
                  plot char(100),
                  star char(40))
               """)
print('完成表的创建,开始插入数据->')#下面开始插入数据
for i in movies:
    cursor.execute("insert into douban(title,ranks,director,release_date,plot,star) values(?,?,?,?,?,?)",(i['title'],i['rank'],i['director'],
                   i['release_date'],i['plot'],i['star']))
print('插入数据完成')
cursor.execute("drop table douban")
cursor.close()
con.commit()
con.close()
qy.close()

连接成功->
开始创建表->
完成表的创建,开始插入数据->
插入数据完成
